<a href="https://colab.research.google.com/github/wiendewani/Image-Retrieval/blob/master/3%20Model%20CNN/Image_Retrieval_MobileNetv2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import tensorflow as tf
from sklearn.neighbors import NearestNeighbors
from multiprocessing import Pool
from skimage.transform import resize
import os
import skimage.io
from multiprocessing import Pool
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import offsetbox
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from sklearn import manifold
import random

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install scikit-image 

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import offsetbox
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from sklearn import manifold

class CV_plot_utils:
  # Plot image
  def plot_img(img, range=[0, 255]):
      plt.imshow(img, vmin=range[0], vmax=range[1])
      plt.xlabel("xpixels")
      plt.ylabel("ypixels")
      plt.tight_layout()
      plt.show()
      plt.close()

  # Plots images in 2 rows: top row is query, bottom row is answer
  def plot_query_retrieval(img_query, imgs_retrieval, outFile):
      n_retrieval = len(imgs_retrieval)
      fig = plt.figure(figsize=(2*n_retrieval, 4))
      fig.suptitle("Image Retrieval (k={})".format(n_retrieval), fontsize=25)

      # Plot query image
      ax = plt.subplot(2, n_retrieval, 0 + 1)
      plt.imshow(img_query)
      ax.get_xaxis().set_visible(False)
      ax.get_yaxis().set_visible(False)
      for axis in ['top', 'bottom', 'left', 'right']:
          ax.spines[axis].set_linewidth(4)  # increase border thickness
          ax.spines[axis].set_color('black')  # set to black
      ax.set_title("query",  fontsize=14)  # set subplot title

      # Plot retrieval images
      for i, img in enumerate(imgs_retrieval):
          ax = plt.subplot(2, n_retrieval, n_retrieval + i + 1)
          plt.imshow(img)
          ax.get_xaxis().set_visible(False)
          ax.get_yaxis().set_visible(False)
          for axis in ['top', 'bottom', 'left', 'right']:
              ax.spines[axis].set_linewidth(1)  # set border thickness
              ax.spines[axis].set_color('black')  # set to black
          ax.set_title("Rank #%d" % (i+1), fontsize=14)  # set subplot title

      if outFile is None:
          plt.show()
      else:
          plt.savefig(outFile, bbox_inches='tight')
      plt.close()

  # Plot t-SNE of images
  def plot_tsne(X, imgs, outFile):

      def imscatter(x, y, images, ax=None, zoom=1.0):
          if ax is None:
              ax = plt.gca()
          x, y = np.atleast_1d(x, y)
          artists = []
          for x0, y0, img0 in zip(x, y, images):
              im = OffsetImage(img0, zoom=zoom)
              ab = AnnotationBbox(im, (x0, y0), xycoords='data', frameon=True)
              artists.append(ax.add_artist(ab))
          ax.update_datalim(np.column_stack([x, y]))
          ax.autoscale()
          return artists

      def plot_embedding(X, imgs, title=None):
          x_min, x_max = np.min(X, 0), np.max(X, 0)
          X = (X - x_min) / (x_max - x_min)

          plt.figure()
          ax = plt.subplot(111)
          for i in range(X.shape[0]):
              plt.text(X[i, 0], X[i, 1], ".", fontdict={'weight': 'bold', 'size': 9})
          if hasattr(offsetbox, 'AnnotationBbox'):
              imscatter(X[:,0], X[:,1], imgs, zoom=0.3, ax=ax)

          plt.xticks([]), plt.yticks([])
          if title is not None:
              plt.title(title, fontsize=18)

      tsne = manifold.TSNE(n_components=2, init='pca', random_state=0)
      X_tsne = tsne.fit_transform(X)
      plot_embedding(X_tsne, imgs, "t-SNE embeddings")
      if outFile is None:
          plt.show()
      else:
          plt.savefig(outFile, bbox_inches='tight')
      plt.close()

  # Plot image reconstructions
  def plot_reconstructions(imgs, imgs_reconstruct, outFile,
                          range_imgs=[0, 255],
                          range_imgs_reconstruct=[0, 1]):
      # Create plot to save
      assert len(imgs) == len(imgs_reconstruct)
      fig = plt.figure(figsize=(20, 4))
      fig.suptitle("Image Reconstructions", fontsize=35)
      n = min(len(imgs), 10)
      for i in range(n):

          # Plot original image
          ax = plt.subplot(2, n, i + 1)
          plt.imshow(imgs[i],
                    vmin=range_imgs[0],
                    vmax=range_imgs[1])
          ax.get_xaxis().set_visible(False)
          ax.get_yaxis().set_visible(False)

          # Plot reconstructed image
          ax = plt.subplot(2, n, n + i + 1)
          plt.imshow(imgs_reconstruct[i],
                    vmin=range_imgs_reconstruct[0],
                    vmax=range_imgs_reconstruct[1])
          ax.get_xaxis().set_visible(False)
          ax.get_yaxis().set_visible(False)

      if outFile is None:
          plt.show()
      else:
          plt.savefig(outFile, bbox_inches='tight')
      plt.close()

In [5]:
import os
import skimage.io
from multiprocessing import Pool

class CV_IO_utils:
  # Read image
  def read_img(filePath):
      return skimage.io.imread(filePath, as_gray=False)

  # Read images with common extensions from a directory
  def read_imgs_dir(dirPath, extensions, parallel=True):
      args = [os.path.join(dirPath, filename)
              for filename in os.listdir(dirPath)
              if any(filename.lower().endswith(ext) for ext in extensions)]
      if parallel:
          pool = Pool()
          imgs = pool.map(CV_IO_utils.read_img, args)
          pool.close()
          pool.join()
      else:
          imgs = [read_img(arg) for arg in args]
      return imgs

  # Save image to file
  def save_img(filePath, img):
      skimage.io.imsave(filePath, img)

In [6]:
from multiprocessing import Pool
from skimage.transform import resize

# Apply transformations to multiple images
class CV_transform_utils:
    
  def apply_transformer(imgs, transformer, parallel=True):
      if parallel:
          pool = Pool()
          imgs_transform = pool.map(transformer, [img for img in imgs])
          pool.close()
          pool.join()
      else:
          imgs_transform = [transformer(img) for img in imgs]
      return imgs_transform

  # Normalize image data [0, 255] -> [0.0, 1.0]
  def normalize_img(img):
      return img / 255.

  # Resize image
  def resize_img(img, shape_resized):
      img_resized = resize(img, shape_resized,
                          anti_aliasing=True,
                          preserve_range=True)
      assert img_resized.shape == shape_resized
      return img_resized

  # Flatten image
  def flatten_img(img):
      return img.flatten("C")

In [7]:
import random
import numpy as np

class utils:
    # Get split indices
  def split(fracs, N, seed):
      fracs = [round(frac, 2) for frac in fracs]
      if sum(fracs) != 1.00:
          raise Exception("fracs do not sum to one!")

      # Shuffle ordered indices
      indices = list(range(N))
      random.Random(seed).shuffle(indices)
      indices = np.array(indices, dtype=int)

      # Get numbers per group
      n_fracs = []
      for i in range(len(fracs) - 1):
          n_fracs.append(int(max(fracs[i] * N, 0)))
      n_fracs.append(int(max(N - sum(n_fracs), 0)))

      if sum(n_fracs) != N:
          raise Exception("n_fracs do not sum to N!")

      # Sample indices
      n_selected = 0
      indices_fracs = []
      for n_frac in n_fracs:
          indices_frac = indices[n_selected:n_selected + n_frac]
          indices_fracs.append(indices_frac)
          n_selected += n_frac

      # Check no intersections
      for a, indices_frac_A in enumerate(indices_fracs):
          for b, indices_frac_B in enumerate(indices_fracs):
              if a == b:
                  continue
              if utils.is_intersect(indices_frac_A, indices_frac_B):
                  raise Exception("there are intersections!")

      return indices_fracs

  # Is there intersection?
  def is_intersect(arr1, arr2):
      n_intersect = len(np.intersect1d(arr1, arr2))
      if n_intersect == 0: return False
      else: return True

In [13]:
import os
import numpy as np
import tensorflow as tf
from sklearn.neighbors import NearestNeighbors
import copy
import sklearn.metrics as metric
from statistics import mean
import re
from tensorflow.keras.applications import MobileNetV2

correct = 0
wrong = 0
loss_ = []
acc_ = []
ground_truth = []
retrieval_labels = []
prec_list = []
rec_list = []

acc_transfer = []
loss_transfer = []
prec_transfer = []
rec_transfer = []


retrieval_label=os.listdir(r"/content/drive/MyDrive/Kembali Citra Notebook/corel_dataset/training_set")
truth_label=os.listdir(r"/content/drive/MyDrive/Kembali Citra Notebook/corel_dataset/test_set")

# Apply transformations to all images
class ImageTransformer(object):

    def __init__(self, shape_resize):
        self.shape_resize = shape_resize

    def __call__(self, img):
        img_transformed = CV_transform_utils.resize_img(img, self.shape_resize)
        img_transformed = CV_transform_utils.normalize_img(img_transformed)
        return img_transformed

transfer = ['Xception', 'VGG16', 'VGG19', 'ResNet50', 'ResNet101', 
            'ResNet152', 'ResNet50V2', 'ResNet101V2', 'ResNet152V2',
            'InceptionV3', 'InceptionResNetV2', 'MobileNet', 'MobileNetV2',
            'DenseNet121', 'DenseNet169', 'DenseNet201', 'NASNetMobile',
            'NASNetLarge', 'EfficientNetB0', 'EfficientNetB1', 'EfficientNetB2',
            'EfficientNetB3', 'EfficientNetB4', 'EfficientNetB5', 'EfficientNetB6',
            'EfficientNetB7']



# Run mode: (autoencoder -> simpleAE, convAE) or (transfer learning -> vgg19)
modelName = "vgg19"  # try: "simpleAE", "convAE", "vgg19"
trainModel = True
parallel = True  # use multicore processing

# Make paths
dataTrainDir = os.path.join(os.getcwd(), "/content/drive/MyDrive/Kembali Citra Notebook/corel_dataset/training_set")
dataTestDir = os.path.join(os.getcwd(), "/content/drive/MyDrive/Kembali Citra Notebook/corel_dataset/test_set")
outDir = os.path.join(os.getcwd(), "output", modelName)
if not os.path.exists(outDir):
    os.makedirs(outDir)

# Read images
extensions = [".jpg", ".jpeg"]
print("Reading train images from '{}'...".format(dataTrainDir))
imgs_train = CV_IO_utils.read_imgs_dir(dataTrainDir, extensions, parallel=parallel)
print("Reading test images from '{}'...".format(dataTestDir))
imgs_test = CV_IO_utils.read_imgs_dir(dataTestDir, extensions, parallel=parallel)
shape_img = imgs_train[0].shape
print("Image shape = {}".format(shape_img))




Reading train images from '/content/drive/MyDrive/Kembali Citra Notebook/corel_dataset/training_set'...
Reading test images from '/content/drive/MyDrive/Kembali Citra Notebook/corel_dataset/test_set'...
Image shape = (256, 384, 3)


In [ ]:
imgs_train[0].shape

(256, 384, 3)

In [ ]:
import skimage as sk

In [ ]:
imgs_train

In [14]:

print("Loading MobileNetV2 pre-trained model...")
model = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False,
                                            input_shape=shape_img)
model.summary()

shape_img_resize = tuple([int(x) for x in model.input.shape[1:]])
input_shape_model = tuple([int(x) for x in model.input.shape[1:]])
output_shape_model = tuple([int(x) for x in model.output.shape[1:]])
n_epochs = None
    # Build models




# Print some model info
print("input_shape_model = {}".format(input_shape_model))
print("output_shape_model = {}".format(output_shape_model))


transformer = ImageTransformer(shape_img_resize)
print("Applying image transformer to training images...")
imgs_train_transformed = CV_transform_utils.apply_transformer(imgs_train, transformer, parallel=parallel)
print("Applying image transformer to test images...")
imgs_test_transformed = CV_transform_utils.apply_transformer(imgs_test, transformer, parallel=parallel)

# Convert images to numpy array
X_train = np.array(imgs_train_transformed).reshape((-1,) + input_shape_model)
X_test = np.array(imgs_test_transformed).reshape((-1,) + input_shape_model)
print(" -> X_train.shape = {}".format(X_train.shape))
print(" -> X_test.shape = {}".format(X_test.shape))




# Create embeddings using model
print("Inferencing embeddings using pre-trained model...")
E_train = model.predict(X_train)
E_train_flatten = E_train.reshape((-1, np.prod(output_shape_model)))
E_test = model.predict(X_test)
E_test_flatten = E_test.reshape((-1, np.prod(output_shape_model)))
print(" -> E_train.shape = {}".format(E_train.shape))
print(" -> E_test.shape = {}".format(E_test.shape))
print(" -> E_train_flatten.shape = {}".format(E_train_flatten.shape))
print(" -> E_test_flatten.shape = {}".format(E_test_flatten.shape))


# Fit kNN model on training images
print("Fitting k-nearest-neighbour model on training images...")
knn = NearestNeighbors(n_neighbors=10, metric="cosine")
knn.fit(E_train_flatten)

# Perform image retrieval on test images
print("Performing image retrieval on test images...")
for i, emb_flatten in enumerate(E_test_flatten):
    _, indices = knn.kneighbors([emb_flatten]) # find k nearest train neighbours
    img_query = imgs_test[i] # query image
    imgs_retrieval = [imgs_train[idx] for idx in indices.flatten()] # retrieval images
    for x in indices.flatten():
      if truth_label[i][:3] in retrieval_label[x]:
        correct = correct + 1
      else:
        wrong = wrong + 1
    loss_.append((wrong/5))
    acc_.append((correct/5))
    correct = 0
    wrong = 0

    # Precision Recall
    for x in indices.flatten():
      ground_truth.append(re.split("(\d.*)", truth_label[i])[0])
      retrieval_labels.append(re.split("(\d.*)", retrieval_label[x])[0])
    
    rec_list.append(metric.recall_score(ground_truth, retrieval_labels, average='weighted'))
    prec_list.append(metric.precision_score(ground_truth, retrieval_labels, average='weighted'))

    ground_truth.clear()
    retrieval_labels.clear()

    outFile = os.path.join(outDir, "{}_retrieval_{}.png".format(modelName, i))
    CV_plot_utils.plot_query_retrieval(img_query, imgs_retrieval, outFile)

print("retrieval accuracy: {}".format(mean(acc_)))
print("retrieval loss: {}".format(mean(loss_)))
print("retrieval precision: {}".format(mean(prec_list)))
print("retrieval recall: {}".format(mean(rec_list)))

Loading MobileNetV2 pre-trained model...
Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 256, 384, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 128, 192, 32  864         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 128, 192, 32  128         ['Conv1[0][0]']                  
                                )     

In [ ]:
model.save('/content/drive/MyDrive/Kembali Citra Notebook/IR.h5')

In [ ]:
model.save_weights('/content/drive/MyDrive/Kembali Citra Notebook/model_weights.h5')

In [ ]:
dataTrainDir = os.path.join(os.getcwd(), "/content/drive/MyDrive/Kembali Citra Notebook/corel_dataset/training_set")
dataTestDir = os.path.join(os.getcwd(), "/content/drive/MyDrive/Kembali Citra Notebook/corel_dataset/test_set")

In [ ]:
from tensorflow.keras.preprocessing.image import img_to_array

In [ ]:
import cv2
import numpy as np

# Gather data train
train_data = []
train_label = []
for r, d, f in os.walk(dataTrainDir):
    for file in f:
        if ".jpg" in file:
            imagePath = os.path.join(r, file)
            image = cv2.imread(imagePath, cv2.IMREAD_COLOR)
            image = cv2.resize(image, (256, 384))
            train_data.append(np.array(image))
            label = imagePath.split(os.path.sep)[-2]
            train_label.append(label)

train_data = np.array(train_data)
train_label = np.array(train_label)

In [ ]:
train_data.shape

(900, 384, 256, 3)

In [ ]:
np.save('data.npy', train_data)

In [ ]:
import numpy as np
print(np.asarray(train_data).shape)
# train_data.shape

(900, 384, 256, 3)


In [ ]:
!pip install fe

In [ ]:
x = np.expand_dims(x_train, axis=0)

In [ ]:
!pip install feature-extractor

ERROR: Could not find a version that satisfies the requirement feature-extractor (from versions: none)
ERROR: No matching distribution found for feature-extractor


In [ ]:
# Import the libraries
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.models import Model
from pathlib import Path
from PIL import Image
import numpy as np

class FeatureExtractor:
    def __init__(self):
        # Use VGG-16 as the architecture and ImageNet for the weight
        base_model = VGG16(weights='imagenet')
        # Customize the model to return features from fully-connected layer
        self.model = Model(inputs=base_model.input, outputs=base_model.get_layer('fc1').output)
    def extract(self, img):
        # Resize the image
        img = img.resize((256, 384))
        # Convert the image color space
        img = img.convert('RGB')
        # Reformat the image
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        # Extract Features
        feature = self.model.predict(x)[0]
        return feature / np.linalg.norm(feature)
# Iterate through images (Change the path based on your image location)
fe = FeatureExtractor()
for img_path in sorted(Path("/content/drive/MyDrive/Kembali Citra Notebook/corel_dataset/training_set").glob("*.jpg")):
    print(img_path)
    # Extract Features
    feature = fe.extract(img=Image.open(img_path))
    # Save the Numpy array (.npy) on designated path
    feature_path = "data.npy"
    np.save(feature_path, feature)

/content/drive/MyDrive/Kembali Citra Notebook/corel_dataset/training_set/10.jpg


ValueError: ignored

In [ ]:
# Import the libraries
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.models import Model
from pathlib import Path
from PIL import Image
from tensorflow.keras.models import load_model



class FeatureExtractor:
    def __init__(self, model):
        self.model = model
    def extract(self, img):
        # Resize the image
        img = img.resize((384,256))
        # Convert the image color space
        img = img.convert('RGB')
        # Reformat the image
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        # Extract Features
        feature = self.model.predict(x)[0]
        return feature / np.linalg.norm(feature)
        
# Iterate through images (Change the path based on your image location)
fe = FeatureExtractor(load_model('/content/drive/MyDrive/Kembali Citra Notebook/IR.h5', compile=False))

for img_path in sorted(Path("/content/drive/MyDrive/Kembali Citra Notebook/corel_dataset/training_set").glob("*.jpg")):    
    print(img_path)
    # Extract Features
    feature = fe.extract(img=Image.open(img_path))
    # Save the Numpy array (.npy) on designated path
    feature_path = "features.npy"
    np.save(feature_path, feature)

/content/drive/MyDrive/Kembali Citra Notebook/corel_dataset/training_set/10.jpg
/content/drive/MyDrive/Kembali Citra Notebook/corel_dataset/training_set/11.jpg
/content/drive/MyDrive/Kembali Citra Notebook/corel_dataset/training_set/110.jpg
/content/drive/MyDrive/Kembali Citra Notebook/corel_dataset/training_set/111.jpg
/content/drive/MyDrive/Kembali Citra Notebook/corel_dataset/training_set/112.jpg
/content/drive/MyDrive/Kembali Citra Notebook/corel_dataset/training_set/113.jpg
/content/drive/MyDrive/Kembali Citra Notebook/corel_dataset/training_set/114.jpg
/content/drive/MyDrive/Kembali Citra Notebook/corel_dataset/training_set/115.jpg
/content/drive/MyDrive/Kembali Citra Notebook/corel_dataset/training_set/116.jpg
/content/drive/MyDrive/Kembali Citra Notebook/corel_dataset/training_set/117.jpg
/content/drive/MyDrive/Kembali Citra Notebook/corel_dataset/training_set/118.jpg
/content/drive/MyDrive/Kembali Citra Notebook/corel_dataset/training_set/119.jpg
/content/drive/MyDrive/Kembali